In [6]:
# imports
import sys
sys.path.insert(1, '/Users/danielalemos/si/src/si/io')
sys.path.insert(1, '/Users/danielalemos/si/src/si/model_selection')
sys.path.insert(1, '/Users/danielalemos/si/src/si/linear_model')

from csv_file import read_csv
from logistic_regression import LogisticRegression
from cross_validate import cross_validate
from grid_search import grid_search_cv

In [7]:
# datasets
breast_bin_dataset = read_csv('/Users/danielalemos/si/datasets/breast-bin.csv', features=False, label=True)

In [8]:
# standardization
from sklearn.preprocessing import StandardScaler
breast_bin_dataset.X = StandardScaler().fit_transform(breast_bin_dataset.X)
breast_bin_dataset.X

array([[ 0.20885295, -0.69912815, -0.74242297, ..., -1.0000359 ,
        -0.61132565, -0.34418721],
       [-0.8578253 , -0.69912815, -0.74242297, ..., -0.58991542,
        -0.61132565, -0.34418721],
       [-0.8578253 , -0.69912815, -0.74242297, ..., -0.17979494,
        -0.61132565, -0.34418721],
       ...,
       [ 0.20885295, -0.37139715, -0.40592217, ..., -1.0000359 ,
        -0.61132565,  0.23892607],
       [-0.8578253 , -0.04366616, -0.40592217, ..., -0.17979494,
        -0.61132565, -0.34418721],
       [ 0.91997179,  0.93952681,  0.94008103, ...,  1.46068699,
        -0.61132565, -0.34418721]])

In [9]:
# cross validation
lg = LogisticRegression()
scores = cross_validate(lg, breast_bin_dataset, cv=5)
scores

{'seeds': [700, 555, 572, 70, 768],
 'train': [0.964221824686941,
  0.9695885509838998,
  0.9731663685152058,
  0.964221824686941,
  0.964221824686941],
 'test': [0.9784172661870504,
  0.9568345323741008,
  0.9424460431654677,
  0.9784172661870504,
  0.9784172661870504]}

In [10]:
# grid search cv

lg = LogisticRegression()

# parameter grid
parameter_grid = {
    'l2_penalty': (1, 10),
    'alpha': (0.001, 0.0001, 0.00001),
    'max_iter': (1000, 2000, 3000, 4000, 5000, 6000)
}

# cross validate the model
scores = grid_search_cv(lg,
                        breast_bin_dataset,
                        parameter_grid=parameter_grid,
                        cv=3)

scores

[{'seeds': [128, 827, 331],
  'train': [0.9695885509838998, 0.9695885509838998, 0.964221824686941],
  'test': [0.9568345323741008, 0.9568345323741008, 0.9784172661870504],
  'parameters': {'l2_penalty': 1, 'alpha': 0.001, 'max_iter': 1000}},
 {'seeds': [433, 141, 550],
  'train': [0.960644007155635, 0.3434704830053667, 0.3434704830053667],
  'test': [0.9928057553956835, 0.34532374100719426, 0.34532374100719426],
  'parameters': {'l2_penalty': 1, 'alpha': 0.001, 'max_iter': 2000}},
 {'seeds': [965, 116, 45],
  'train': [0.9695885509838998, 0.33810375670840787, 0.3488372093023256],
  'test': [0.9568345323741008, 0.3669064748201439, 0.3237410071942446],
  'parameters': {'l2_penalty': 1, 'alpha': 0.001, 'max_iter': 3000}},
 {'seeds': [612, 421, 700],
  'train': [0.964221824686941, 0.33810375670840787, 0.33273703041144903],
  'test': [0.9784172661870504, 0.3669064748201439, 0.38848920863309355],
  'parameters': {'l2_penalty': 1, 'alpha': 0.001, 'max_iter': 4000}},
 {'seeds': [555, 572, 70],

In [11]:
import pandas as pd
cols = list(scores[0]['parameters'].keys())
cols = cols + ['train', 'test', 'cv']

dict_df = {col: [] for col in cols}
for score in scores:
    for i, (train_val, test_val) in enumerate(zip(score['train'], score['test'])):
        dict_df['cv'].append(i)
        dict_df['train'].append(train_val)
        dict_df['test'].append(test_val)
        for p_key, p_val in score['parameters'].items():
            dict_df[p_key].append(p_val)

df = pd.DataFrame(dict_df)
df

,l2_penalty,alpha,max_iter,train,test,cv
0,1,0.00100,1000,0.969589,0.956835,0
1,1,0.00100,1000,0.969589,0.956835,1
2,1,0.00100,1000,0.964222,0.978417,2
3,1,0.00100,2000,0.960644,0.992806,0
4,1,0.00100,2000,0.343470,0.345324,1
...,...,...,...,...,...,...
103,10,0.00001,5000,0.338104,0.366906,1
104,10,0.00001,5000,0.348837,0.323741,2
105,10,0.00001,6000,0.964222,0.978417,0
106,10,0.00001,6000,0.338104,0.366906,1
